In [38]:
import cv2
from skimage import feature
import matplotlib.pyplot as plt
import numpy as np
import os



In [39]:
def lbp_process(path):
    # Baca citra
    img = cv2.imread(path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Hitung LBP
    radius = 1
    n_points = 8 * radius
    lbp_img = feature.local_binary_pattern(gray_img, n_points, radius, method="uniform")

    lbp_hist, _ = np.histogram(lbp_img.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-6)
    
    return lbp_hist

In [40]:
# Direktori awal untuk memulai iterasi
root_directory = '../detect-lamp/Unit/res/'  # Sesuaikan dengan direktori awal Anda
row = []


for foldername, subfolders, filenames in os.walk(root_directory):
    for filename in filenames:
        # print(foldername)
        path = os.path.join(foldername, filename)
        lbp = lbp_process(path)
        if(foldername.split('/')[-1] == 'nyala'):
            lbp = np.append(lbp, 1)
        else:
            lbp = np.append(lbp, 0)
        row.append(lbp)

In [41]:
row

[array([0.07563862, 0.07959339, 0.0346555 , 0.05144129, 0.0676412 ,
        0.05806187, 0.04432271, 0.07285564, 0.36750059, 0.1482892 ,
        0.        ]),
 array([0.07881584, 0.08804306, 0.04152249, 0.06862745, 0.10803537,
        0.07958478, 0.04267589, 0.07670127, 0.27124183, 0.14475202,
        0.        ]),
 array([0.07255003, 0.07893375, 0.03306878, 0.05279503, 0.06958822,
        0.05992639, 0.04471475, 0.07401656, 0.36890384, 0.14550265,
        0.        ]),
 array([0.07570073, 0.08000441, 0.03189141, 0.05291326, 0.06670713,
        0.05821011, 0.03878835, 0.07421099, 0.38297285, 0.13860075,
        0.        ]),
 array([0.06505539, 0.06706674, 0.02611448, 0.04088631, 0.05305328,
        0.05163545, 0.03129122, 0.05991163, 0.4893168 , 0.11566869,
        0.        ]),
 array([0.06434043, 0.07004255, 0.02340426, 0.0493617 , 0.06323404,
        0.05761702, 0.03234043, 0.05965957, 0.46714894, 0.11285106,
        0.        ]),
 array([0.0691586 , 0.08292978, 0.02996368, 0.061289

In [42]:
import pandas as pd

In [43]:
# Daftar nama kolom
column_names = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "label"]

# Membuat DataFrame
df = pd.DataFrame(row, columns=column_names)
df.head()

,1,2,3,4,5,6,7,8,9,10,label
0,0.075639,0.079593,0.034655,0.051441,0.067641,0.058062,0.044323,0.072856,0.367501,0.148289,0.0
1,0.078816,0.088043,0.041522,0.068627,0.108035,0.079585,0.042676,0.076701,0.271242,0.144752,0.0
2,0.072550,0.078934,0.033069,0.052795,0.069588,0.059926,0.044715,0.074017,0.368904,0.145503,0.0
3,0.075701,0.080004,0.031891,0.052913,0.066707,0.058210,0.038788,0.074211,0.382973,0.138601,0.0
4,0.065055,0.067067,0.026114,0.040886,0.053053,0.051635,0.031291,0.059912,0.489317,0.115669,0.0


In [44]:
df[df['label'] == 1.0]

,1,2,3,4,5,6,7,8,9,10,label


In [45]:
y = df['label']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

In [46]:
X = df.iloc[:,:-1]
X.head()

,1,2,3,4,5,6,7,8,9,10
0,0.075639,0.079593,0.034655,0.051441,0.067641,0.058062,0.044323,0.072856,0.367501,0.148289
1,0.078816,0.088043,0.041522,0.068627,0.108035,0.079585,0.042676,0.076701,0.271242,0.144752
2,0.072550,0.078934,0.033069,0.052795,0.069588,0.059926,0.044715,0.074017,0.368904,0.145503
3,0.075701,0.080004,0.031891,0.052913,0.066707,0.058210,0.038788,0.074211,0.382973,0.138601
4,0.065055,0.067067,0.026114,0.040886,0.053053,0.051635,0.031291,0.059912,0.489317,0.115669


In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
import xgboost as xgb

In [49]:
model = xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss")

In [50]:
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [51]:
y_pred = model.predict(X_test)


In [52]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 1.0


In [53]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Data latihan (X_train, y_train) dan data pengujian (X_test, y_test) sudah ada

# Buat objek XGBoost
xgb_model = xgb.XGBClassifier()

# Definisikan grid parameter yang akan diuji
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

# Buat objek GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

# Lakukan grid search
grid_search.fit(X_train, y_train)

# Hasil terbaik
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Gunakan model dengan parameter terbaik pada data pengujian
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)


Fitting 3 folds for each of 243 candidates, totalling 729 fits


Best Parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Best Score: 1.0


In [54]:
model.save_model("without-roi-lampu.model")

C:\Users\whisn\AppData\Roaming\Python\Python39\site-packages\xgboost\core.py:160: UserWarning: [12:26:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
